# IBM Applied Data Science Capstone Notebook
This notebook will be mainly used for the capstone project.

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Segmenting and Clustering Neighborhoods in Toronto
Import liberies for scraping wikipedia page.

In [3]:
import requests
from bs4 import BeautifulSoup

Scrape the Wikipedia page for the Toronto neighborhood data.

In [4]:
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
website_url = requests.get(wiki).text
soup = BeautifulSoup(website_url,'lxml')

Store the table from Wikipedia page into 3 lists: Postcode, Borough, Neighbourhood.

In [5]:
Postcode = []
Borough = []
Neighbourhood = []
for items in soup.find('table', class_='sortable').find_all('tr')[1::1]:
    data = items.find_all(['td'])
    try:
        Postcode.append(data[0].text)
        Borough.append(data[1].text)
        Neighbourhood.append(data[2].text.strip())
    except IndexError:pass

Merge the lists into a pandas dataframe, remove all the data with Borough value is "Not assigned" and then show the first 5 rows of the dataframe.

In [6]:
df = pd.DataFrame({'Postcode' : Postcode,
                    'Borough' : Borough,
                    'Neighbourhood' : Neighbourhood})
df = df[df['Borough'] !='Not assigned'].reset_index(drop=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Goup the records by Postcode and Borough, join the Neighbourhood with same Postcode, show the number of records in the new dataframe.

In [7]:
aggregation_functions = { 'Borough': 'first', 'Neighbourhood': ', '.join}
df_new = df.groupby(df['Postcode']).aggregate(aggregation_functions).reset_index()
df_new.loc[df_new['Neighbourhood'] =='Not assigned','Neighbourhood'] = df_new['Borough']
df_new.shape

(103, 3)